In [ ]:
from typing import TypedDict
from typing_extensions import Annotated
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph,START,END
from langgraph.prebuilt import ToolNode,tools_condition
from langchain.chat_models import init_chat_model
from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId,tool

from langgraph.types import Command,interrupt


In [7]:
@tool
def human_assistance(name:str,birthday:str,tool_call_id:Annotated[str,InjectedToolCallId]):
    """Request assistance from a human"""
    human_response=interrupt({
        "question": "Is this correct",
        "name":name,
        "birthday":birthday
    })
    
    if human_response.get("correct","").lower().startswith("y"):
        verified_name=name
        verified_birthday=birthday
        response="Correct"
        
    else:
        verified_name=human_response.get("name",name)
        verified_birthday = human_response.get("birthday", birthday)
        response = f"Made a correction: {human_response}"
    
    state_update={
        "name": verified_name,
        "birthday": verified_birthday,
        "messages": [ToolMessage(response,tool_call_id=tool_call_id)]
    }
    
    return Command(update=state_update)

In [ ]:
class State(TypedDict):
    messages: Annotated[list,add_messages]
    name:str
    birthday:str
    
def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

graph_builder = StateGraph(State)
tools=[human_assistance]
llm= init_chat_model("gpt-4.1-mini").bind_tools(tools)


graph_builder.add_node("chatbot",chatbot)

graph_builder.add_edge(START,"chatbot")

tool_node=ToolNod
